In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import plotly.express as px

from src.consts import PATHS


In [25]:
MAX_FEATURES_TITLE = '⅓ lowest knockout'
MIN_FEATURES_TITLE = '⅓ highest knockout'

In [26]:


def get_ds(model_size: str):
    df = pd.read_csv(f'/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-{model_size}-hf/evaluate_context_interference/ds=counter_fact/{model_size}_norm_1_output.csv')
    n_layers = df.layer.max()
    df = df[(df['layer'] < (n_layers-9))]
    df = df[df['category'] != 'mid']
    df['category'] = df['category'].str.replace('max', MAX_FEATURES_TITLE).str.replace('min', MIN_FEATURES_TITLE)
    df['acc'] = 100*(df['acc'])
    return df

In [27]:




def plot_feature_knockout(model_size: str):
    df = get_ds(model_size)

    fig = px.line(df, y='acc', x='layer', color='category', line_dash='category',
                color_discrete_map={MAX_FEATURES_TITLE:'forestgreen', MIN_FEATURES_TITLE:'purple'},
                line_dash_map={MAX_FEATURES_TITLE:'solid', MIN_FEATURES_TITLE:'dashdot'},)
    fig.update_layout(title='Feature knockout', 
                    xaxis_title='Layer', 
                    yaxis_title='Relative change in probability', 
                    template='plotly_white',
                    legend_title='',
                    font=dict(size=18, family='Arial'),
                    width=800, height=400)

    fig.add_hline(y=0, line_dash='dot', line_color='black')
    fig.show()
    fig.write_image(PATHS.RESULTS_DIR / f'feature_knockout_{model_size}.png', scale=4)

In [37]:
import matplotlib.pyplot as plt


def plot_feature_knockout(model_size: str):
    df = get_ds(model_size)  # Load the data

    # Define line colors and styles
    colors = {
        MAX_FEATURES_TITLE: 'forestgreen',
        MIN_FEATURES_TITLE: 'purple',
    }
    line_styles = {
        MAX_FEATURES_TITLE: 'solid',
        MIN_FEATURES_TITLE: 'dashdot',
    }

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(5, 3))

    # Group data by 'category' and plot each group
    for category, group in df.groupby('category'):
        ax.plot(
            group['layer'],
            group['acc'],
            label=category,
            color=colors[category],
            linestyle=line_styles[category],
            linewidth=2
        )

    # Add a horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='dotted', linewidth=1)

    # Customize plot appearance
    plt.suptitle(
        f'Mamba1-{model_size}'
        '\n'
        'Feature knockout based on L1 norm'
        , fontsize=12)
    ax.set_xlabel('Layer', fontsize=14)
    ax.set_ylabel('Probability change', fontsize=14)

    # Customize ticks font size
    ax.tick_params(axis='both', labelsize=12)

    ax.legend(
        fontsize=12,  # Adjust font size
        frameon=False,  # Remove legend border for cleaner appearance
    )

    # Add grid for better visualization
    ax.grid(True, linestyle='--', linewidth=0.5)

    # Save and show the figure
    plt.tight_layout()
    plt.savefig(PATHS.RESULTS_DIR / f'feature_knockout_mamba1_{model_size}.png')
    plt.close(fig)


for model_size in ['2.8B', '1.4B', '130M']:
    plot_feature_knockout(model_size)

In [1]:
@dataclass
class Args:
    # model_arch: MODEL_ARCH = MODEL_ARCH.MINIMAL_MAMBA2_new
    model_arch: MODEL_ARCH = MODEL_ARCH.MAMBA1
    model_size: str = "2.8B"
    dataset_args: DatasetArgs = pyrallis.field(
        default=DatasetArgs(name=DATASETS.COUNTER_FACT, splits="all"), is_mutable=True
    )
    filteration: str = FILTERATIONS.all_correct
    _batch_size: int = 16  # Adjust based on GPU memory
    output_file: Optional[Path] = None
    with_slurm: bool = False
    temperature = 1
    top_k = 0
    top_p = 1
    window_size = 5
    prompt_indices = [1,2,3,4,5]
    knockout_map = {'last': ['last', 'first', "subject", "relation"], 
                    'subject': ['context', 'subject']}

    output_dir: Optional[Path] = None

    @property
    def batch_size(self) -> int:
        return (
            1
            if (
                self.model_arch == MODEL_ARCH.MINIMAL_MAMBA2
                or self.model_arch == MODEL_ARCH.MINIMAL_MAMBA2_new
            )
            else self._batch_size
        )

    @property
    def model_id(self) -> TModelID:
        return MODEL_SIZES_PER_ARCH_TO_MODEL_ID[self.model_arch][self.model_size]


NameError: name 'dataclass' is not defined

In [5]:
args = Args()

In [6]:

original_res, attn_res = [
    pd.read_parquet(
        PATHS.OUTPUT_DIR
        / args.model_id
        / "data_construction"
        / f"ds={args.dataset_args.dataset_name}"
        / f"entire_results_{"attention" if attention else "original"}.parquet"
    )
    for attention in [True, False]
]

mask = (original_res["hit"] == attn_res["hit"]) & (attn_res["hit"] == True)
data = attn_res[mask]


In [7]:
data

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,original_idx,split,true_prob,max_prob,hit,pred
0,{} is a product of,,{} is a product of,Windows XP Media Center Edition is a product of,P178,Q312,Q2283,Microsoft,Apple,Windows XP Media Center Edition,9209,train1,0.726621,0.726621,True,Microsoft
1,"In {}, the language spoken is",In,", the language spoken is","In United Kingdom, the language spoken is",P37,Q1412,Q1860,English,Finnish,United Kingdom,18566,train1,0.529818,0.529818,True,English
2,{} is a native speaker of,,{} is a native speaker of,Henry de Montherlant is a native speaker of,P103,Q1860,Q150,French,English,Henry de Montherlant,6916,train1,0.748545,0.748545,True,French
3,The native language of {} is,The native language of,is,The native language of Olga Georges-Picot is,P103,Q7737,Q150,French,Russian,Olga Georges-Picot,140,train1,0.184752,0.184752,True,French
4,{} is owned by,,{} is owned by,Google Marketing Platform is owned by,P127,Q183,Q95,Google,Germany,Google Marketing Platform,2720,train1,0.757935,0.757935,True,Google
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,{} is created by,,{} is created by,Microsoft InfoPath is created by,P178,Q312,Q2283,Microsoft,Apple,Microsoft InfoPath,15035,test,0.425025,0.425025,True,Microsoft
823,{} worked in the city of,,{} worked in the city of,Pavlo Skoropadskyi worked in the city of,P937,Q84,Q1899,Kiev,London,Pavlo Skoropadskyi,18585,test,0.134802,0.134802,True,K
824,"{}, created by",,"{}, created by","IBM 3790, created by",P176,Q66,Q37156,IBM,Boeing,IBM 3790,19782,test,0.523892,0.523892,True,IBM
825,{} belongs to the continent of,,{} belongs to the continent of,Riiser-Larsen Ice Shelf belongs to the contine...,P30,Q48,Q51,Antarctica,Asia,Riiser-Larsen Ice Shelf,9105,test,0.820605,0.820605,True,Antar


In [ ]:
df = pd.read_csv('/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-2.8B-hf/evaluate_context_interference/ds=counter_fact/2.8B_norm_1_output.csv')
print(df)

In [14]:
import pandas as pd

df = pd.read_csv('/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-2.8B-hf/evaluate_context_interference/ds=counter_fact/2.8B_norm_1_output.csv')
fig = px.line(df, x='layer', y='acc', color='category')
fig.update_layout(title='')
fig.show()

In [15]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-1.4B-hf/evaluate_context_interference/ds=counter_fact/1.4B_norm_1_output.csv')
px.line(df, x='layer', y='acc', color='category').show()

In [16]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-130M-hf/evaluate_context_interference/ds=counter_fact/130M_norm_1_output.csv')
px.line(df, x='layer', y='acc', color='category').show()

In [ ]:
from typing import Iterable, List

import torch
from torch import Tensor, matmul, zeros_like

from .. import KnockoutMode


def knockout_scan(seq_len: int, ssm_state: Tensor, discrete_A: Tensor, discrete_B: Tensor, u: Tensor, C: Tensor, knocked_out_inputs: Iterable[int], affected_outputs: Iterable[int], knockout_mode: KnockoutMode, dtype) -> List[Tensor]:
    deltaB_u = discrete_B * u[:, :, :, None].float()
    knockout_state = zeros_like(ssm_state)
    scan_outputs = []
    for i in range(seq_len):
        ssm_state = discrete_A[:, :, i, :] * ssm_state + deltaB_u[:, :, i, :]      # [batch, intermediade_size, ssm_state]
        if i not in knocked_out_inputs:
            knockout_state = discrete_A[:, :, i, :] * knockout_state + deltaB_u[:, :, i, :]      # [batch, intermediade_size, ssm_state]
        elif i in knocked_out_inputs:
            if knockout_mode == KnockoutMode.ZERO_ATTENTION:
                knockout_state = discrete_A[:, :, i, :] * knockout_state
            elif knockout_mode == KnockoutMode.ZERO_DELTA:
                knockout_state = knockout_state
        if i in affected_outputs:
            scan_output = matmul(knockout_state.to(dtype), C[:, i, :].unsqueeze(-1))  # [batch, intermediade_size, 1]
        else:
            scan_output = matmul(ssm_state.to(dtype), C[:, i, :].unsqueeze(-1))  # [batch, intermediade_size, 1]
        scan_outputs.append(scan_output[:, :, 0])

    return scan_outputs


def materialize_ssm_transition(A: torch.Tensor) -> torch.Tensor:
    batch = A.shape[0]
    D = A.shape[1]
    T = A.shape[2]
    N = A.shape[3]
    A = A.transpose(-1,-2).repeat(1,1,1,T).reshape(batch,D,N,T,T).transpose(-1,-2)
    A = torch.tril(A) + torch.triu(torch.ones_like(A),1)
    A_cumprod = torch.cumprod(A, dim=-2)

    transition_mat = A_cumprod.transpose(-2,-3)

    return transition_mat


def materialize_ssm_attention(A: Tensor, B: Tensor, C: Tensor, return_transition: bool) -> torch.Tensor | tuple[torch.Tensor, torch.Tensor]: 
    transition_mat = materialize_ssm_transition(A)

    AB = (transition_mat * B.unsqueeze(-1))

    out = torch.einsum('btn, bdtnq -> bdtq', C, AB)
    
    if return_transition:
        return out, transition_mat
    
    return out


def knockout_matrix(seq_len: int, discrete_A: Tensor, discrete_B: Tensor, u: Tensor, C: Tensor, knocked_out_inputs: Iterable[int], affected_outputs: Iterable[int], dtype) -> List[Tensor]:
    attn = materialize_ssm_attention(discrete_A, discrete_B, C, False)
    for i, j in zip(affected_outputs, knocked_out_inputs):
        attn[:, :, i, j] = 0
    outputs = attn * u[:, :, :, None].float()
    return outputs


In [8]:
import pandas as pd

df = pd.read_parquet("/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-2.8B-hf/data_construction/ds=counter_fact/entire_results_original.parquet")
df['target_true'].head()

0     Microsoft
1       English
2        French
3        French
4        Google
Name: target_true, dtype: object

In [2]:
import json

# Open and read the JSON file
with open('/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-2.8B-hf/info_flow_test_top_outputs/ds=counter_fact/ws=9/block_last_target_last/outputs.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data)

[[[[9664, ' Microsoft', 0.9600555300712585], [7464, ' Windows', 0.0322045236825943], [253, ' the', 0.006631913594901562], [16880, 'Microsoft', 0.0005786402616649866], [14877, 'Windows', 0.00013698582188226283]]], [[[4383, ' English', 0.9996795654296875], [4782, ' British', 0.00014863394608255476], [6571, ' mostly', 6.665463297395036e-05], [7194, ' mainly', 4.3976102460874245e-05], [1925, ' called', 3.1884770578471944e-05]]], [[[5112, ' French', 0.9879401922225952], [4383, ' English', 0.01107520516961813], [1097, ' both', 0.0003751036711037159], [9883, ' Spanish', 0.00026353111024945974], [253, ' the', 0.00014860212104395032]]], [[[5112, ' French', 0.7840160727500916], [4383, ' English', 0.10410340130329132], [417, ' not', 0.0661514550447464], [7202, ' unknown', 0.035282522439956665], [1335, ' still', 0.0022929527331143618]]], [[[5559, ' Google', 0.9987107515335083], [17664, 'Google', 0.0008913984056562185], [17899, ' google', 0.0002078302059089765], [3186, ' search', 9.959286398952827e

In [56]:
df

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,original_idx,split,true_prob,max_prob,hit,pred
0,{} is a product of,,{} is a product of,Windows XP Media Center Edition is a product of,P178,Q312,Q2283,Microsoft,Apple,Windows XP Media Center Edition,9209,train1,0.726621,0.726621,True,Microsoft
1,"In {}, the language spoken is",In,", the language spoken is","In United Kingdom, the language spoken is",P37,Q1412,Q1860,English,Finnish,United Kingdom,18566,train1,0.529818,0.529818,True,English
2,{} is a native speaker of,,{} is a native speaker of,Henry de Montherlant is a native speaker of,P103,Q1860,Q150,French,English,Henry de Montherlant,6916,train1,0.748545,0.748545,True,French
3,The native language of {} is,The native language of,is,The native language of Olga Georges-Picot is,P103,Q7737,Q150,French,Russian,Olga Georges-Picot,140,train1,0.184752,0.184752,True,French
4,{} is owned by,,{} is owned by,Google Marketing Platform is owned by,P127,Q183,Q95,Google,Germany,Google Marketing Platform,2720,train1,0.757935,0.757935,True,Google
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,{} is created by,,{} is created by,Microsoft InfoPath is created by,P178,Q312,Q2283,Microsoft,Apple,Microsoft InfoPath,15035,test,0.425025,0.425025,True,Microsoft
823,{} worked in the city of,,{} worked in the city of,Pavlo Skoropadskyi worked in the city of,P937,Q84,Q1899,Kiev,London,Pavlo Skoropadskyi,18585,test,0.134802,0.134802,True,K
824,"{}, created by",,"{}, created by","IBM 3790, created by",P176,Q66,Q37156,IBM,Boeing,IBM 3790,19782,test,0.523892,0.523892,True,IBM
825,{} belongs to the continent of,,{} belongs to the continent of,Riiser-Larsen Ice Shelf belongs to the contine...,P30,Q48,Q51,Antarctica,Asia,Riiser-Larsen Ice Shelf,9105,test,0.820605,0.820605,True,Antar


In [105]:
new_df = {
    # 'index': [],
    'row': [],
    'token': [],
    'prob': [],
    'rank': [],
    'token_id': []
}

t = 0 

for i in range(len(data)):
    for j, opt in enumerate(data[i][0]):
        # new_df['index'].append(t)
        new_df['row'].append(i)
        new_df['rank'].append(j)
        new_df['token'].append(opt[1])
        new_df['prob'].append(opt[2])
        new_df['token_id'].append(opt[0])
        t += 1

print(new_df)

new_df = pd.DataFrame(new_df)


{'row': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 5

In [197]:
from src.consts import (
    MODEL_SIZES_PER_ARCH_TO_MODEL_ID,
    PATHS,
)

# "/a/home/cc/students/cs/nirendy/repos/ssm_analysis/output/state-spaces/mamba-2.8B-hf/data_construction/ds=counter_fact/entire_results_original.parquet"
original_res, attn_res = [
    pd.read_parquet(
        PATHS.OUTPUT_DIR
        / 'state-spaces/mamba-2.8B-hf'
        / "data_construction"
        / "ds=counter_fact"
        / f"entire_results_{"attention" if attention else "original"}.parquet"
    )
    for attention in [True, False]
]

mask = (original_res["hit"] == attn_res["hit"]) & (attn_res["hit"] == True)
data = attn_res[mask]
data = data.reset_index(drop=True)


In [198]:
new_df = new_df[new_df['rank'] == 0]
merged = data.merge(new_df, how='left', left_index=True, right_on=['row'])

In [199]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [200]:
merged['correct'] = merged.apply(lambda row: row['token_id'] == tokenizer(row['target_true'])['input_ids'][0], axis=1)
merged['correct'].sum()

762

In [201]:
filtered = merged[merged['correct']]
filtered['prob diff'] = filtered['prob'] - filtered['true_prob']
filtered['normalized prob diff'] = filtered['prob diff'] / filtered['true_prob']
filtered.rename(columns={'true_prob': 'baseline prob', 'prob': 'knockout prob'}, inplace=True)
print(filtered.shape)

(762, 24)


/tmp/ipykernel_4047827/4164167411.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_4047827/4164167411.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_4047827/4164167411.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [210]:
import plotly.express as px

px.scatter(filtered, x='baseline prob', y='knockout prob').show()
# merged.loc[0, 'token']

In [211]:

px.scatter(filtered, x='baseline prob', y='prob diff').show()

In [212]:


px.scatter(filtered, x='baseline prob', y='normalized prob diff').show()

In [213]:
px.histogram(filtered, x='normalized prob diff').show()

In [ ]:

px.histogram(filtered, x='baseline prob').show()

In [207]:
filtered.sort_values('normalized prob diff', ascending= False)[['normalized prob diff', 'prob diff', 'baseline prob', 'knockout prob']].corr()

,normalized prob diff,prob diff,baseline prob,knockout prob
normalized prob diff,1.000000,0.788688,-0.776852,-0.065352
prob diff,0.788688,1.000000,-0.820757,0.198883
baseline prob,-0.776852,-0.820757,1.000000,0.396631
knockout prob,-0.065352,0.198883,0.396631,1.000000


In [208]:
filtered.sort_values('normalized prob diff')
filtered[['normalized prob diff', 'baseline prob', 'prob diff', ]].corr()

,normalized prob diff,baseline prob,prob diff
normalized prob diff,1.000000,-0.776852,0.788688
baseline prob,-0.776852,1.000000,-0.820757
prob diff,0.788688,-0.820757,1.000000


In [214]:
high_base_prob_filtered = filtered[filtered['baseline prob'] >= 0.5]
px.histogram(high_base_prob_filtered, x='normalized prob diff').show()

In [ ]:
high_base_prob_filtered = filtered[filtered['baseline prob'] >= 0.5]

norm_prob = high_base_prob_filtered['normalized prob diff'].to_numpy()
norm_prob.sort()

mean = norm_prob.cumsum() / (np.arange(norm_prob.shape[0]) + 1)

px.scatter(x=np.arange(norm_prob.shape[0]), y=mean)

In [168]:
merged['prefix_len'] = merged['relation_prefix'].str.len()

In [171]:
(merged['prefix_len'] == 0).sum()

670

In [217]:
filtered

,relation,relation_prefix,relation_suffix,prompt,relation_id,target_false_id,target_true_id,target_true,target_false,subject,...,hit,pred,row,token,knockout prob,rank,token_id,correct,prob diff,normalized prob diff
0,{} is a product of,,{} is a product of,Windows XP Media Center Edition is a product of,P178,Q312,Q2283,Microsoft,Apple,Windows XP Media Center Edition,...,True,Microsoft,0,Microsoft,0.960056,0,9664,True,0.233435,0.321261
5,"In {}, the language spoken is",In,", the language spoken is","In United Kingdom, the language spoken is",P37,Q1412,Q1860,English,Finnish,United Kingdom,...,True,English,1,English,0.999680,0,4383,True,0.469861,0.886835
10,{} is a native speaker of,,{} is a native speaker of,Henry de Montherlant is a native speaker of,P103,Q1860,Q150,French,English,Henry de Montherlant,...,True,French,2,French,0.987940,0,5112,True,0.239395,0.319814
15,The native language of {} is,The native language of,is,The native language of Olga Georges-Picot is,P103,Q7737,Q150,French,Russian,Olga Georges-Picot,...,True,French,3,French,0.784016,0,5112,True,0.599264,3.243615
20,{} is owned by,,{} is owned by,Google Marketing Platform is owned by,P127,Q183,Q95,Google,Germany,Google Marketing Platform,...,True,Google,4,Google,0.998711,0,5559,True,0.240776,0.317674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4025,{} is created by,,{} is created by,Microsoft InfoPath is created by,P178,Q312,Q2283,Microsoft,Apple,Microsoft InfoPath,...,True,Microsoft,805,Microsoft,0.994226,0,9664,True,0.569201,1.339217
4030,{} worked in the city of,,{} worked in the city of,Pavlo Skoropadskyi worked in the city of,P937,Q84,Q1899,Kiev,London,Pavlo Skoropadskyi,...,True,K,806,K,0.220406,0,611,True,0.085604,0.635038
4035,"{}, created by",,"{}, created by","IBM 3790, created by",P176,Q66,Q37156,IBM,Boeing,IBM 3790,...,True,IBM,807,IBM,0.983616,0,21314,True,0.459724,0.877517
4040,{} belongs to the continent of,,{} belongs to the continent of,Riiser-Larsen Ice Shelf belongs to the contine...,P30,Q48,Q51,Antarctica,Asia,Riiser-Larsen Ice Shelf,...,True,Antar,808,Antar,0.994120,0,31913,True,0.173516,0.211449


In [218]:
filtered['relation_len'] = filtered['relation_suffix'].str.len()

/tmp/ipykernel_4047827/2794215132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [227]:
(filtered['relation_len'] >= 4).sum()

644

In [176]:

px.histogram(filtered[(filtered['prefix_len'] > 0 )], x='normalized prob diff').show()

In [178]:
px.histogram(data, x='true_prob')

In [180]:
(data['true_prob'] < 0.5).sum()

285